In [109]:
import math
import numpy as np
import math
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import re
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import requests
from wikidata.client import Client
from itertools import islice
import ast
import csv
from tqdm import tqdm

In [5]:
#Save the two dataset to csv
silverdataset = pd.read_csv('datasets/silver.csv')
golddataset = pd.read_csv('datasets/gold.csv')

silver_links = pd.read_csv('datasets/wikipedia_links_silver.csv')
gold_links = pd.read_csv('datasets/wikipedia_links_gold.csv')

In [ ]:
#Extract totality of languages
silver_lang = set(silver_links['language'])
gold_lang = set(gold_links['language'].unique())
total_lang_count = silver_lang.union(gold_lang)

348


In [19]:
#Group entities to construct dictionary
# 🔁 Group by 'entity', crea un dizionario {lang: url} per ciascuna entity
df_silver = pd.DataFrame(silver_links)
df_gold = pd.DataFrame(gold_links)

create_dic = lambda x: dict(zip(x["language"], x["url"]))
silver_grouped = df_silver.groupby("entity").apply(create_dic).reset_index(name="lang_url_map")
gold_grouped = df_gold.groupby("entity").apply(create_dic).reset_index(name="lang_url_map")

silver_grouped.to_csv("datasets/grouped_silver_links.csv", index=False)
gold_grouped.to_csv("datasets/grouped_gold_links.csv", index=False)

/var/folders/yq/gyf11wn125qgkzlcv24mbp3w0000gn/T/ipykernel_74220/1917892312.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  silver_grouped = df_silver.groupby("entity").apply(create_dic).reset_index(name="lang_url_map")
/var/folders/yq/gyf11wn125qgkzlcv24mbp3w0000gn/T/ipykernel_74220/1917892312.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gold_grouped = df_gold.groupby("entity").apply(create_dic)

In [41]:
head = silver_grouped.head(1)
for idx, row in head.iterrows():
    results = [""]     
    lang_distribution = []
    for key in list(dict.keys()):
        text, length = extract_text(key, dict[key])
        if (key == 'en'): #the language is english -> i can save the text for further studies
            results[0] = text
        lang_distribution.append(length)

    #append variance and avg to results
    l = len(lang_distribution)
    mean = sum(lang_distribution)/l
    results.append(mean)
    results.append(math.sqrt(sum((x - mean) ** 2 for x in lang_distribution) / l))  # Popolazione)
    print(results)



["Vincent Lange (born 15 June 1974 in Berlin) actor, producer, sports manager and coach, ex volleyball player from Germany, who competed for the Men's National Team in the 2000s. He played as a receiver/spiker and libero.\n\n\n== Honours ==\n(Beach Volleyball Pro / International Tournaments & Top Rankings 1997 - 2001)\n2001 European Championship — 9th place\n2002 FIVB World League — 9th place\n(German Champion 2003 / SCC Berlin)\n(European Cup Champion 2006 / Copra Berni Piacenza)\n\n\n== References ==\n\n\n== External links ==\nVincent Lange at FIVB.com \nVincent Lange at the Beach Volleyball Database", 758.25, 512.6525992326577]


In [ ]:
def extract_text(lang, wikipedia_url):
    
    title = wikipedia_url.split('/')[-1]
    #print("english wikipedia title:", title)
    lang = lang.replace('_', '-')

    # Use Wikipedia's API to get plain text of the article
    api_url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "titles": title,
        "format": "json",
        "redirects": 1
    }

    try:
        res = requests.get(api_url, params=params, timeout=3).json()
        page = next(iter(res["query"]["pages"].values())) #see below the dictionary, we create an iterable and pick the first and only item
        text = page.get("extract", "") # we get the "extract field"
        if lang == 'en':
            return (text, len(text))
        return ("", len(text))
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta: {e} per link {wikipedia_url}")
        return ("", 0)

In [104]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

# Funzione per salvare progressivamente i risultati in un file CSV
def save_wikipedia_text(file_name):
    output_file_name = "wikipedia_text_stats_"+file_name
    # Controlla se il file di output esiste già
    if os.path.exists(output_file_name):
        # Carica il file esistente
        existing_df = pd.read_csv(output_file_name)
        # Raccoglie le keyword già trattate
        processed_keywords = existing_df['entity'].unique().tolist()
        print(f"🌍 {len(processed_keywords)} keywords già trattate.")
    else:
        existing_df = pd.DataFrame(columns=['entity', 'engtext', 'distribution', 'std', 'avg'])
        processed_keywords = []
    
    # Carica il dataset di input
    data = pd.read_csv("datasets/"+file_name)
    df = pd.DataFrame(data, columns=['entity', 'lang_url_map']) #refers to the wikipedia url

    # Lista per raccogliere i risultati
    results = [None, ""]

    # Cicla su ogni entità
    for idx, row in df.iterrows():
        #print(row)
        item = row['entity']
        dict = ast.literal_eval(row['lang_url_map']) #ast converts string to dictionary
        results[0] = item
        
        # Se la keyword è già stata trattata, salta
        if item in processed_keywords:
            print(f"🔁 '{item}' già trattata, salto.")
            continue
        print(f"Elaboro: {item}")

        lang_distribution = []
        for key in list(dict.keys()):
            #print("working with: ", key, dict[key])
            text, length = extract_text(key, dict[key])
            if length == 0:
                continue
            if (key == 'en'): #the language is english -> i can save the text for further studies
                clean_text = text.replace('\n', ' ').replace('\t', ' ').strip()
                results[1] = [clean_text]
            lang_distribution.append(length)

        #append lang_distribution
        results.append(lang_distribution)

        #append std and avg to results
        l = len(lang_distribution)
        mean = sum(lang_distribution)/l
        results.append(math.sqrt(sum((x - mean) ** 2 for x in lang_distribution) / l))  # Popolazione)
        results.append(mean)
        
        # Scrivi progressivamente i risultati
        with open(output_file_name, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            if not os.path.exists(output_file_name):
                writer.writerow(['entity', 'engtext', 'distribution', 'std', 'avg'])  # o dinamico, se serve
            writer.writerow(results)
        results = [None, ""]  # Resetta la lista dei risultati per non aggiungere più volte
        
        # Aggiungi la keyword alla lista delle trattate
        processed_keywords.append(item)

    print(f"✅ Processamento completato. Risultati salvati in '{output_file_name}'.")


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_entity(row, processed_keywords):
    item = row['entity']
    dict_url = ast.literal_eval(row['lang_url_map'])

    if item in processed_keywords:
        return None

    print(f"⚙️ Processando '{item}'")
    results = [item, ""]
    lang_distribution = []

    for key in dict_url:
        text, length = extract_text(key, dict_url[key])
        if length == 0:
            continue
        if key == 'en':
            clean_text = text.replace('\n', ' ').replace('\t', ' ').strip()
            results[1] = clean_text
        lang_distribution.append(length)

    if not lang_distribution:
        return None

    l = len(lang_distribution)
    mean = sum(lang_distribution) / l
    std = math.sqrt(sum((x - mean) ** 2 for x in lang_distribution) / l)

    results.append(lang_distribution)
    results.append(std)
    results.append(mean)

    return results

def save_wikipedia_text_parallel(file_name):
    output_file_name = "wikipedia_text_stats_" + file_name

    # Carica keyword già trattate
    if os.path.exists(output_file_name):
        existing_df = pd.read_csv(output_file_name)
        processed_keywords = existing_df['entity'].unique().tolist()
        print(f"🌍 {len(processed_keywords)} keywords già trattate.")
    else:
        processed_keywords = []
        # Scriviamo l'header se il file non esiste
        with open(output_file_name, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['entity', 'engtext', 'distribution', 'std', 'avg'])

    data = pd.read_csv("datasets/" + file_name)
    df = pd.DataFrame(data, columns=['entity', 'lang_url_map'])

    # Parallelizza il processo
    with ProcessPoolExecutor() as executor:
        future_to_entity = {
            executor.submit(process_entity, row, processed_keywords): row['entity']
            for _, row in df.iterrows()
        }

        for future in tqdm(as_completed(future_to_entity), total=len(future_to_entity)):
            result = future.result()
            if result:
                # Scrivi subito sul file ogni risultato ricevuto
                with open(output_file_name, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(result)

    print(f"✅ Processamento completato. Risultati salvati in '{output_file_name}'.")


    


🌍 24 keywords già trattate.


Process SpawnProcess-26:
  0%|          | 0/300 [00:00<?, ?it/s]Process SpawnProcess-25:
Traceback (most recent call last):
Traceback (most recent call last):
Process SpawnProcess-27:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/concurrent/futures/process.py", line 246, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_entity' on <

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
#entity, engtext, distribution, std, avg
if __name__ == '__main__':
    save_wikipedia_text_parallel('grouped_gold_links.csv')
